<a href="https://colab.research.google.com/github/Alroy05/resume-screening/blob/master/resume_parsing_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy-transformers

In [ ]:
!pip install -U spacy

In [4]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 28.19 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [6]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [7]:
len(cv_data)

200

In [8]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg config.cfg

2023-09-02 01:33:10.773330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
cv_data

In [10]:

def get_spacy_doc(file,data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text,annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start,end,label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db




In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=1)

In [12]:
len(train), len(test)

(199, 1)

In [13]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 1/1 [00:00<00:00, 89.79it/s]


In [14]:
len(db.tokens)

1

In [17]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-09-02 01:34:26.208039: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were

In [20]:
nlp = spacy.load('/content/output/model-best')

In [24]:
doc = nlp('my name is Zane Falcao. I worked at Microsoft. I have 3 years of experience')

for ent in doc.ents:
   print(ent.text, " ->>>>>", ent.label_)

Zane Falcao  ->>>>> Name
Microsoft  ->>>>> Companies worked at
3 years  ->>>>> Years of Experience
